In [8]:
import folium
import pandas as pd
import pickle
import os
import h3
from label_samples_time_hexa import label_samples
from vizualize import *
from func import *

In [9]:
matrix = pd.read_pickle("/home/jaro/BINP29/Project_Eran/1_dist_matrix/eucl_dist.pkl")

In [10]:
time_bins = 11
resolution = 3
same_age_range = True
df = label_samples("/home/jaro/BINP29/Project_Eran/", time_bins, resolution, same_age_range)
time_bins = rename_time_bins(df)
time_bins_hexagons = get_time_bin_hexagons(df)

In [11]:
time_bins_dist = calc_dist_time_bin(df, matrix)

In [12]:
selected_time_bin = time_bins[5]
hexagons = time_bins_hexagons[selected_time_bin]
time_bin = time_bins_dist[selected_time_bin]
threshold = 0.4

In [13]:
time_bin = scale_distances(time_bin)

In [14]:
lines = get_distance_lines(time_bin)
m = draw_sample_hexagons(hexagons)
m = draw_barriers(lines, m)
m

In [41]:
threshold = 0.4
isolated_hex, barrier_lines, barrier_hex = get_isolated_hex_and_barriers(time_bin, hexagons, threshold, allowed_distance=10)

print(f"Number of isolated hexagons: {len(isolated_hex)}")
print(f"Number of barrier lines: {len(barrier_lines)}")
print(f"Number of barrier hexagons: {len(barrier_hex)}")

83182afffffffff
831854fffffffff
831851fffffffff
83185efffffffff
831858fffffffff
83390afffffffff
833908fffffffff
833974fffffffff
831f20fffffffff
832c34fffffffff
832d1cfffffffff
832c28fffffffff
832c2efffffffff
832c04fffffffff
832c31fffffffff
832c33fffffffff
832d52fffffffff
8315b5fffffffff
8315a6fffffffff
83254dfffffffff
83256afffffffff
832563fffffffff
8310aefffffffff
832d6bfffffffff
832d69fffffffff
83119afffffffff
831e21fffffffff
831e2cfffffffff
831f0bfffffffff
831f01fffffffff
831f09fffffffff
831f76fffffffff
831f29fffffffff
831e1cfffffffff
83184dfffffffff
83186bfffffffff
83184cfffffffff
831e20fffffffff
831e23fffffffff
831e00fffffffff
832da6fffffffff
832da3fffffffff
833f6bfffffffff
833f6afffffffff
833f4cfffffffff
833f44fffffffff
833f71fffffffff
831ee4fffffffff
831ee5fffffffff
831e09fffffffff
831193fffffffff
832d6cfffffffff
832d6efffffffff
832d6afffffffff
832d41fffffffff
832d43fffffffff
833954fffffffff
833973fffffffff
833945fffffffff
83394cfffffffff
83394dfffffffff
833961fffffffff
831f92ff

In [42]:
# get the index of the time bin of interest
time_bin_index = time_bins.index(time_bins[5])
# get the closest population for each isolated hexagon
closest_populations, new_isolated_hex = find_closest_population(df, time_bin_index, isolated_hex, matrix, threshold)
print(f"Number of isolated hexagons with no migration: {len(new_isolated_hex)}")

Number of isolated hexagons with no migration: 5


In [43]:
imputed_hex_k = impute_missing_hexagons(barrier_hex)
print(len(imputed_hex_k))

765


In [33]:
m = draw_sample_hexagons(hexagons)
m = draw_hexagons(new_isolated_hex, color = "red", m = m, opacity=0.7, value="No migration")
m = draw_hexagons_with_values(barrier_hex, m, threshold = 0.0)
m = draw_hexagons_with_values(imputed_hex, m, threshold = 0.0, imputed=True)
m = draw_barriers(barrier_lines, m)
m = draw_migration_for_time_bin(closest_populations, m)
m